### 練習問題1.44
関数の平滑化(smoothing)は、  
信号処理において重要な概念である。  
$f$が関数で$dx$がある⼩さな値であるとき、  
$f$を平滑化したものは$x$での値が$f(x - dx), f(x), f(x + dx)$の平均となる関数である。  
⼊⼒として$f$を計算する⼿続きを取り、$f$を平滑化したものを計算する⼿続きを返す⼿続きsmoothを書け。  
時には、関数を繰り返し平滑化する(つまり、平滑化した関数をさらに平滑化することを続ける)ことによって**$n$重平滑化関数**(n-fold smoothedfunction)を得ることが役に⽴つこともある。  
smoothと練習問題1.43のrepeatedを使って任意の関数の$n$重平滑化関数を⽣成するやり⽅を⽰せ。

In [1]:
(define (compose f g)
  (lambda (x)(f (g x)))
  )
(define (repeated f n)
  (define (iter g count)
          (if (>= count n) g
              (iter (compose f g) (+ count 1))
          )
    )
  (iter f 1)
  )

(define (smoothing f dx)
  (lambda (x)
    (let ((x0 (f (- x dx)))
          (x1 (f x))
          (x2 (f (+ x dx))))
      (/ (+ x0 x1 x2) 3.0)
      )
    )
  )

; 回答
; repeated手続きを使った回答になっていない。
; repeated手続きを使うと期待した動作にならなかった。

; これは正しくない。
; fを平滑化した関数を、繰り返し合成すると、
; fの出力をさらに合成することにあるので、別物の関数になる。
; fを繰り返し合成してはいけない。
;(define (smoothing-n f n dx)
;  (repeated (smoothing f dx) n))

; repeated手続きを使用しない方法
; この動作は正しい。
;(define (smoothing-n f n dx)
;  (define (iter i g)
;    (if (= i 0) g
;        (iter (- i 1) (smoothing g dx))
;        )
;    )
;  (iter n f)
;  )

; repeated手続きを使った回答
(define (smoothing-n f n dx)
  (if (= n 0) f  ; n=0のとき平滑化なし
      (let ((g (lambda (func)(smoothing func dx))))
        (let ((r (repeated g n))) ; n重平滑化の手続き
          (r f) ; fをn重平滑化
          )
      )
    )
  )

In [2]:
(define (f x)
  (cond ((< x 20) 0)
        ((and (>= x 20) (< x 40)) (- (* 5 x) 100))
        ((and (>= x 40) (< x 60)) 100)
        ((and (>= x 60) (< x 80)) (+ (* -5 x) 400))
        ((>= x 80) 0)
    )
  )

In [3]:
; (x, y)を縦に出力する
(define (disp-map proc items)
  (if (null? items) '()
       (let ((x (car items)))
         (let ((y (proc x)))
           (begin
            (display x)
            (display "\t")
            (display y)
            (newline)
            (disp-map proc (cdr items))
            )
           )
        )
    )
  )

(define (make-list n m)
  (if (> n m) (list)
      (append (list n) (make-list (+ n 1) m))
    )
  )

(define x-domain (make-list 0 100))
;x-domain

; 縦に出力する場合
;(disp-map f x-domain)

(map f x-domain)

(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 95 90 85 80 75 70 65 60 55 50 45 40 35 30 25 20 15 10 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)

In [4]:
(define g (smoothing f 5))
(map g x-domain)

(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.6666666666666667 3.3333333333333335 5.0 6.666666666666667 8.333333333333334 11.666666666666666 15.0 18.333333333333332 21.666666666666668 25.0 30.0 35.0 40.0 45.0 50.0 55.0 60.0 65.0 70.0 75.0 78.33333333333333 81.66666666666667 85.0 88.33333333333333 91.66666666666667 93.33333333333333 95.0 96.66666666666667 98.33333333333333 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 98.33333333333333 96.66666666666667 95.0 93.33333333333333 91.66666666666667 88.33333333333333 85.0 81.66666666666667 78.33333333333333 75.0 70.0 65.0 60.0 55.0 50.0 45.0 40.0 35.0 30.0 25.0 21.666666666666668 18.333333333333332 15.0 11.666666666666666 8.333333333333334 6.666666666666667 5.0 3.3333333333333335 1.6666666666666667 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)

In [5]:
(define h (smoothing-n f 4 5))
(map h x-domain)

(0.0 0.0617283950617284 0.1234567901234568 0.1851851851851852 0.2469135802469136 0.308641975308642 0.6172839506172838 0.9259259259259257 1.234567901234568 1.5432098765432098 1.8518518518518523 2.7777777777777772 3.703703703703703 4.62962962962963 5.555555555555556 6.481481481481484 8.395061728395062 10.30864197530864 12.222222222222221 14.135802469135804 16.049382716049383 19.074074074074073 22.098765432098762 25.123456790123456 28.14814814814815 31.17283950617284 34.93827160493827 38.7037037037037 42.46913580246913 46.23456790123456 50.0 53.76543209876542 57.53086419753087 61.296296296296305 65.06172839506172 68.82716049382718 71.85185185185185 74.87654320987656 77.90123456790126 80.92592592592592 83.95061728395063 85.80246913580247 87.65432098765433 89.50617283950619 91.35802469135803 93.20987654320989 93.82716049382715 94.44444444444446 95.06172839506173 95.67901234567903 96.2962962962963 95.67901234567903 95.06172839506173 94.44444444444446 93.82716049382715 93.20987654320989 91.35

In [6]:
(define gg (smoothing f 1))
(map gg x-domain)

(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.6666666666666667 5.0 10.0 15.0 20.0 25.0 30.0 35.0 40.0 45.0 50.0 55.0 60.0 65.0 70.0 75.0 80.0 85.0 90.0 95.0 98.33333333333333 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 98.33333333333333 95.0 90.0 85.0 80.0 75.0 70.0 65.0 60.0 55.0 50.0 45.0 40.0 35.0 30.0 25.0 20.0 15.0 10.0 5.0 1.6666666666666667 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)

In [7]:
(define hh (smoothing-n f 4 1))
(map hh x-domain)

(0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0617283950617284 0.3703703703703704 1.2962962962962965 3.2098765432098766 6.296296296296297 10.37037037037037 15.061728395061728 20.0 25.0 30.0 35.0 40.0 45.0 50.0 55.0 60.0 65.0 70.0 75.0 80.0 84.93827160493828 89.62962962962963 93.70370370370371 96.79012345679013 98.70370370370371 99.62962962962963 99.93827160493828 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 100.0 99.93827160493828 99.62962962962963 98.70370370370371 96.79012345679013 93.70370370370371 89.62962962962963 84.93827160493828 80.0 75.0 70.0 65.0 60.0 55.0 50.0 45.0 40.0 35.0 30.0 25.0 20.0 15.061728395061728 10.37037037037037 6.296296296296297 3.2098765432098766 1.2962962962962965 0.3703703703703704 0.0617283950617284 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0)